# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 63 new papers today
          24 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/23 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.13208


extracting tarball to tmp_2301.13208...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.13214


extracting tarball to tmp_2301.13214...

...

✔ → 38225:\section{NIRSpec Science Performance}
  ↳ 66327:\section{NIRSpec Science Calibration Pipeline}\label{sec:pipeline}
✔ → 66327:\section{NIRSpec Science Calibration Pipeline}\label{sec:pipeline}
  ↳ 70897:\section{Summary and Conclusions}


✔ → 70897:\section{Summary and Conclusions}
  ↳ 74095:end


Retrieving document from  https://arxiv.org/e-print/2301.13810


extracting tarball to tmp_2301.13810...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.13526-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.13526) | **Extended neutral hydrogen filamentary network in NGC 2403**  |
|| S. Veronese, W. J. G. de Blok, <mark>F. Walter</mark> |
|*Appeared on*| *2023-02-01*|
|*Comments*| **|
|**Abstract**| We present new neutral hydrogen (HI) observations of the nearby galaxy NGC 2403 to determine the nature of a low-column density cloud that was detected earlier by the Green Bank Telescope. We find that this cloud is the tip of a complex of filaments of extraplanar gas that is coincident with the thin disk. The total HI mass of the complex is $2\times10^{7}\text{ M}_\odot$ or 0.6% of the total HI mass of the galaxy. The main structure, previously referred to as the 8-kpc filament, is now seen to be even more extended, along a 20 kpc stream. The kinematics and morphological properties of the filaments are unlikely to be the result of outflows related to galactic fountains. It is more likely that the 20 kpc filament is related to a recent galaxy interaction. In this context, a $\sim$ 50 kpc long stellar stream has been recently detected connecting NGC 2403 with the nearby dwarf satellite DDO 44. Intriguingly, the southern tip of this stream overlaps with that of 20 kpc HI filament. We conclude that the HII anomalies in NGC 2403 are the result of a recent ($\sim2\text{ Gyr}$) interaction with DDO 44 leading to the observed filamentary complex. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.13766-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.13766) | **In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the  James Webb Space Telescope**  |
|| T. Böker, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2023-02-01*|
|*Comments*| *accepted by PASP for special issue on JWST in-orbit performance*|
|**Abstract**| The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectro-photometric and wavelength calibration of NIRSpec spectra, and provide the as measured sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.13214-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.13214) | **The extended "stellar halo" of the Ursa Minor dwarf galaxy**  |
|| Federico Sestito, et al. -- incl., <mark>Lina D'Aoust</mark> |
|*Appeared on*| *2023-02-01*|
|*Comments*| *Submitted to MNRAS, 17 pages, 10 figures*|
|**Abstract**| Five stars in the extreme outskirts (from $\sim5$ to $\sim12$ elliptical half-light radii, r$_h$) of the Ursa Minor (UMi) dwarf galaxy have been identified as potential new members using a Bayesian algorithm applied to \textit{Gaia} EDR3 data. These targets were observed with the GRACES spectrograph, resulting in precise radial velocities and metallicities that confirm their association with UMi. For the brightest and outermost star (Target~1, at $\sim12$ r$_h$), the chemical abundances of $\alpha$- (Mg, Ca, Ti), odd-Z (Na, K, Sc), Fe-peak (Fe, Ni, Cr), and neutron-capture process (Ba) elements have also been determined. We also discuss data from the literature and from APOGEE DR17. We find the chemical patterns in UMi are consistent with a star formation history that includes yields from core collapse supernovae, asymptotic giant branch stars, and supernovae Ia. Evidence for a knee in the [$\alpha$/Fe] ratios near [Fe/H] $\sim-2.1$ indicates a low star formation efficiency similar to that in other dwarf galaxies. Given the distance of Target~1 from the centre of UMi (R$\sim$4.5 kpc), we show that UMi has a more extended structure than previously thought. This "stellar halo" around UMi could be a secondary feature resulting from tidal stripping after multiple orbits around the Galaxy, or maybe a primary UMi feature due to early hierarchical accretion activity or to strong gravitational fluctuations prompted by feedback in the early star formation phase. Also consistent with observations is a late-time merger-free scenario where outside-in star formation is accompanied by gradual supernovae Ia enrichment. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.13651-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.13651) | **Radio Study of the Pulsar Wind Nebula Powered by PSR B1706-44**  |
|| <mark>Y. H. Liu</mark>, C.-Y. Ng, R. Dodson |
|*Appeared on*| *2023-02-01*|
|*Comments*| *16 pages, 9 figures, accepted by APJ*|
|**Abstract**| PSR B1706$-$44 is an energetic gamma-ray pulsar located inside supernova remnant (SNR) G343.1$-$2.3 and it powers a compact pulsar wind nebula (PWN) that shows torus and jet structure in X-rays. We present a radio study of the PWN using Australia Telescope Compact Array (ATCA) observations at 3, 6, 13, and 21\,cm. We found an overall arc-like morphology at 3 and 6\,cm, and the ``arc" shows two distinct peaks at 6\,cm. The radio emission is faint inside the X-ray PWN and only brightens beyond that. We develop a thick torus model with Doppler boosting effect to explain the radio PWN structure. The model suggests a bulk flow speed of $\sim 0.2c$, which could indicate significant deceleration of the flow from the X-ray emitting region. Our polarization result reveals a highly ordered toroidal $B$-field in the PWN. Its origin is unclear given that the supernova reverse shock should have interacted with the PWN. At a larger scale, the 13 and 21\,cm radio images detected a semi-circular rim and an east-west ridge of G343.1$-$2.3. We argue that the latter could possibly be a pulsar tail rather than a filament of the SNR, as supported by the flat radio spectrum and the alignment between the magnetic field and its elongation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.13810-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.13810) | **A Study of Nine Triply Eclipsing Triples**  |
|| S.A. Rappaport, et al. -- incl., <mark>G. Torres</mark> |
|*Appeared on*| *2023-02-01*|
|*Comments*| *34 pages, 17 figures, 24 tables, accepted for publication to MNRAS*|
|**Abstract**| In this work we report the independent discovery and analysis of nine new compact triply eclipsing triple star systems found with the TESS mission: TICs 47151245, 81525800, 99013269, 229785001, 276162169, 280883908, 294803663, 332521671, and 356324779. Each of these nine systems exhibits distinct third-body eclipses where the third (`tertiary') star occults the inner eclipsing binary (EB), or vice versa. We utilize a photodynamical analysis of the TESS photometry, archival photometric data, TESS eclipse timing variations of the EBs, available archival spectral energy distribution curves (SED), and, in some cases, newly acquired radial velocity observations, to solve for the parameters of all three stars, as well as most of the orbital elements. From these analyses we find that the outer orbits of all nine systems are viewed nearly edge on (i.e., within $\lesssim 4^\circ$), and 6 of the systems are coplanar to within $5^\circ$; the others have mutual inclination angles of $20^\circ$, $41^\circ$, and possibly $179^\circ$ (i.e., a retrograde outer orbit). The outer orbital periods range from 47.8 days to 604 days, with eccentricities spanning 0.004 to 0.61. The masses of all 18 EB stars are in the range of 0.9-2.6 M$_\odot$ and are mostly situated near the main sequence. By contrast, the masses and radii of the tertiary stars range from 1.4-2.8 M$_\odot$ and 1.5-13 R$_\odot$, respectively. We make use of the system parameters from these 9 systems, plus those from a comparable number of compact triply eclipsing triples published previously, to gain some statistical insight into their properties. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.13221-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.13221) | **Universal gravity-driven isothermal turbulence cascade in disk galaxies**  |
|| Jérémy Fensch, et al. -- incl., <mark>Patrick Hennebelle</mark> |
|*Appeared on*| *2023-02-01*|
|*Comments*| *12 pages, 13 figures. Accepted for publication in A&A*|
|**Abstract**| While interstellar gas is known to be supersonically turbulent, the injection processes of this turbulence are still unclear. Many studies suggest a dominant role of gravitational instabilities. However, their effect on galaxy morphology and large-scale dynamics vary across cosmic times, in particular due to the evolution of the gas fraction of galaxies. In this paper, we propose numerical simulations to follow the isothermal turbulent cascade of purely gravitationally-driven turbulence from its injection scale down to 0.095 pc for a gas-poor spiral disk and a gas-rich clumpy disk. To this purpose, and to lift the memory-footprint technical lock of sufficiently resolving the interstellar medium of a galaxy, we developed an encapsulated zoom method that allows us to probe self-consistently the self-generated turbulence cascade over three orders of magnitude on spatial scales. We follow this cascade for 10 Myrs. We find that the turbulent cascade follows the same scaling laws in both setups. Namely, in both cases the turbulence is close to equipartition between its compressive and solenoidal modes, the velocity power spectrum follows the Burgers' scaling and the density power spectrum is rather shallow, with a power-law slope of -0.7. Last, gravitationally-bound substructures follow a mass distribution with a -1.8 slope, similar to that of CO clumps. These simulations thus suggest a universality of gravity-driven isothermal turbulent cascade in disk galaxies across cosmic time. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.13226-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.13226) | **A High Precision Survey of the D/H Ratio in the Nearby Interstellar  Medium**  |
|| S. D. Friedman, et al. -- incl., <mark>G. Hebrard</mark> |
|*Appeared on*| *2023-02-01*|
|*Comments*| *37 pages, 10 figures. Accepted for publication in the Astrophysical Journal*|
|**Abstract**| We present high S/N measurements of the H I Ly alpha absorption line toward 16 Galactic targets which are at distances between approximately 190 and 2200 pc, all beyond the wall of the Local Bubble. We describe the models used to remove stellar emission and absorption features and the methods used to account for all known sources of error in order to compute high precision values of the H I column density with robust determinations of uncertainties. When combined with H2 column densities from other sources, we find total H column densities ranging from 10e20.01 to 10e21.25/cm2. Using deuterium column densities from FUSE observations we determine the D/H ratio along the sight lines. We confirm and strengthen the conclusion that D/H is spatially variable over these H I column density and target distance regimes, which predominantly probe the ISM outside the Local Bubble. We discuss how these results affect models of Galactic chemical evolution. We also present an analysis of metal lines along the five sight lines for which we have high resolution spectra and, along with results reported in the literature, discuss the corresponding column densities in the context of a generalized depletion analysis. We find that D/H is only weakly correlated with metal depletion and conclude that the spatial D/H variability is not solely due to dust depletion. A bifurcation of D/Htot as a function of depletion at high depletion levels provides modest support that deuterium-rich gas is infalling onto the Galactic plane. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.13230-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.13230) | **Strong lensing selection effects**  |
|| Alessandro Sonnenfeld, et al. -- incl., <mark>Shun-Sheng Li</mark> |
|*Appeared on*| *2023-02-01*|
|*Comments*| *Submitted to Astronomy & Astrophysics. A two-minute summary video of this paper is available at this https URL*|
|**Abstract**| Context. Strong lenses are a biased subset of the general population of galaxies. Aims. The goal of this work is to quantify how lens galaxies and lensed sources differ from their parent distribution, namely the strong lensing bias. Methods. We first studied how the strong lensing cross-section varies as a function of lens and source properties. Then, we simulated strong lensing surveys with data similar to that expected for Euclid and measured the strong lensing bias in different scenarios. We focused particularly on two quantities: the stellar population synthesis mismatch parameter, $\alpha_{sps}$, defined as the ratio between the true stellar mass of a galaxy and the stellar mass obtained from photometry, and the central dark matter mass at fixed stellar mass and size. Results. Strong lens galaxies are biased towards larger stellar masses, smaller half-mass radii and larger dark matter masses. The amplitude of the bias depends on the intrinsic scatter in the mass-related parameters of the galaxy population and on the completeness in Einstein radius of the lens sample. For values of the scatter that are consistent with observed scaling relations and a minimum detectable Einstein radius of $0.5''$, the strong lensing bias in $\alpha_{sps}$ is $10\%$, while that in the central dark matter mass is $5\%$. The bias has little dependence on the properties of the source population: samples of galaxy-galaxy lenses and galaxy-quasar lenses that probe the same Einstein radius distribution are biased in a very similar way. Quadruply imaged quasar lenses, however, are biased towards higher ellipticity galaxies. Conclusions. Given current uncertainties, strong lensing observations can be used directly to improve our current knowledge of the inner structure of galaxies, without the need to correct for selection effects. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.13526.md
    + _build/html/tmp_2301.13526/./2403_mosaic_5kms_r05_HI_mwfilt_chanmap.png
    + _build/html/tmp_2301.13526/./2403_mosaic_5kms_r05_HI_mwfilt_shuffle_filaments_maps.png
    + _build/html/tmp_2301.13526/./2403_mosaic_5kms_r05_HI_mwfilt_shuffle_pvslice.png
exported in  _build/html/2301.13766.md
    + _build/html/tmp_2301.13766/./Figure_wata.png
    + _build/html/tmp_2301.13766/./light_path_color.png
    + _build/html/tmp_2301.13766/./nirspec_cad_color.png
    + _build/html/tmp_2301.13766/./Figure_msata_cycle1.png
    + _build/html/tmp_2301.13766/./Figure_cycle1_ref.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# Extended neutral hydrogen filamentary network in NGC 2403

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.13526-b31b1b.svg)](https://arxiv.org/abs/2301.13526)<mark>Appeared on: 2023-02-01</mark> - __

</div>
<div id="authors">

Simone Veronese, W. J. G. de Blok, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present new neutral hydrogen (H{\sc i}) observations of the nearby galaxy NGC 2403 to determine the nature of a low-column density cloud that was detected earlier by the Green Bank Telescope.\\Wefind that this cloud is the tip of a complex of filaments of extraplanar gas that is coincident with the thin disk. The total H{\sc i}mass of the complex is$2\times10^{7}\text{ M}_\odot$or 0.6\%of the total H{\sc i}mass of the galaxy. The main structure, previously referred to as the 8-kpc filament, is now seen to be even more extended, along a 20 kpc stream.\\Thekinematics and morphological properties of the filaments are unlikely to be the result of outflows related to galactic fountains. It is more likely that the 20 kpc filament is related to a recent galaxy interaction. In this context, a$\sim$50 kpc long stellar stream has been recently detected connecting NGC 2403 with the nearby dwarf satellite DDO 44. Intriguingly, the southern tip of this stream overlaps with that of 20 kpc H{\sc i}filament.\\Weconclude that the H{\sc i}anomalies in NGC 2403 are the result of a recent ($\sim2\text{ Gyr}$) interaction with DDO 44 leading to the observed filamentary complex.

</div>

<div id="div_fig1">

<img src="tmp_2301.13526/./2403_mosaic_5kms_r05_HI_mwfilt_chanmap.png" alt="Fig2" width="100%"/>

**Figure 2. -** Channel maps of the data cube with both new and archival observations where we show only every second channel. The colour-scale contours correspond to 2 $\times$(1, 4, 16, 64) $\times$ 10\textsuperscript{18} cm\textsuperscript{-2}, where $2\times10^{18}$ cm\textsuperscript{-2} is the $3\sigma$ 1-channel column density limit. The thick contour levels indicate the significant emission identified by the SoFiA source finder. Most of the low-level structures shown in thin contours are noise or mosaicking artefacts. The dashed grey contours denote the column density of $-$2 $\times$ 10\textsuperscript{18} cm\textsuperscript{-2}. The egg-shape borders are due to the 20\% cut-off threshold of the primary beam response. At the bottom we report the line-of-sight velocity (w.r.t. the systemic velocity 133.2 km s$^{-1}$) in each channel. The filamentary detections, labelled with coloured arrows, are evident between $-$59 km s$^{-1}$ and 47 km s$^{-1}$. (*fig:chanmap*)

</div>
<div id="div_fig2">

<img src="tmp_2301.13526/./2403_mosaic_5kms_r05_HI_mwfilt_shuffle_filaments_maps.png" alt="Fig4" width="100%"/>

**Figure 4. -** \textit{Left panel}: primary beam corrected H {\sc i} column density map of NGC 2403 (in reversed grey-scale colormap) overlaid with the GBT cloud candidate (green) from \citet{blok14}. The map was produced by integrating over the channels from $-22$ km s$^{-1}$ to 20 km s$^{-1}$ w.r.t. the systemic velocity. This range corresponds to the one used by \citet{blok14} to compute the candidate GBT cloud moment 0 map. Grey-scale contour levels are from 5 $\times$ 10\textsuperscript{18} cm\textsuperscript{-2} to 1 $\times$ 10\textsuperscript{21} cm\textsuperscript{-2}. The $3\sigma$ 1-channel column density limit is reported at the bottom. The beam of the interferometric data for both the GBT and our VLA observations is shown in the bottom right. The green contours, instead, define the (6.25, 12.5, 25, 62.5) × 10\textsuperscript{17} cm\textsuperscript{-2} column density in the GBT data. The light-grey line denotes the 20\% cut-off threshold of the primary beam response. \textit{Central panel}: primary beam corrected H {\sc i} column density map of the anomalous-velocity gas. Grey-scale contour levels are (3, 10, 25, 50, 100) $\times$ 10\textsuperscript{18} cm\textsuperscript{-2} column density. The spatial scale is the same as in the left panel, as well as the definition of the light grey line. We indicate the galaxy edge with the dashed-black contour. The light blue dashed line represents the path along which the position-velocity slice has been extracted (see Fig. \ref{fig:filpv}). \textit{Right panel}: Illustrative example of the selection effect introduced by the shuffle procedure (see discussion in section \ref{sec:disk}). The map shows the fraction of the original extraplanar flux density of a mock galaxy we were able to recover and the red contour encloses the region where we are severely affected by selection effect, i.e., where more than 90\% of the original flux density is lost. The blanked bits in the centre denote the region where the result is unreliable because of the steep gradient in velocity. The dashed white contours are, instead, the region occupied by the observed filamentary complex. (*fig:filmap*)

</div>
<div id="div_fig3">

<img src="tmp_2301.13526/./2403_mosaic_5kms_r05_HI_mwfilt_shuffle_pvslice.png" alt="Fig3" width="100%"/>

**Figure 3. -** Position-velocity diagram along the major axis (left panel) and minor axis (right panel) through a beam-wide slice of the shuffled data cube without the thin disk removal. The emission is mapped with a reversed grey-scale and we overlaid the 2 $\times$(1, 3, 9) $\times10^{18}$ cm\textsuperscript{-2} levels with grey-scale contours. The blanked area in the top-left and blanked stripe in the bottom are the result of the Milky Way filtering. The grey dashed lines denote the column density of $-3\sigma$. The channels between $\pm$ 65 km s$^{-1}$ with the residual thin disk and diffuse extraplanar emission are highlighted with the hatched area. They were not used for the computation of the moment map shown in the central panel of Fig. \ref{fig:filmap}. (*fig:shufflepv*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\lta}{\lesssim}$
$\newcommand{\gta}{\gtrsim}$
$\newcommand{\coone}{\rm ^{12}CO(1-0)}$
$\newcommand{\cotwo}{\rm ^{12}CO(2-1)}$
$\newcommand{\one}{\rm (1-0)}$
$\newcommand{\two}{\rm (2-1)}$
$\newcommand{\ione}{I_{10}}$
$\newcommand{\itwo}{I_{21}}$
$\newcommand{\nhico}{X_{CO}}$
$\newcommand{\CO}{^{12}CO }$
$\newcommand{\COe}{CO(1-0)}$
$\newcommand{\COz}{CO(2-1) }$
$\newcommand{\solm}{M_{\odot}}$
$\newcommand{\solms}{ {\rm M_{\odot}/pc^2}}$
$\newcommand{\rsol}{R_{\odot}}$
$\newcommand{\rf}{\par\noindent\hangindent 15pt  }$
$\newcommand{\ea}{et al.}$
$\newcommand{\kkms}{ {\rm K} {\rm km} {\rm s}^{-1}}$
$\newcommand{\kms}{ {\rm km} {\rm s}^{-1}}$
$\newcommand{\gyr}{ {\rm Gyr}}$
$\newcommand{\myr}{ {\rm Myr}}$
$\newcommand{\yr}{ {\rm yr}}$
$\newcommand{\s}{ {\rm s}}$
$\newcommand{\pc}{ {\rm pc}}$
$\newcommand{\kpc}{ {\rm kpc}}$
$\newcommand{\mpc}{ {\rm Mpc}}$
$\newcommand{\m}{ {\rm m}}$
$\newcommand{\km}{ {\rm km}}$
$\newcommand{\ergs}{ {\rm erg} {\rm s}^{-1}}$
$\newcommand{\ergsmic}{ {\rm erg} {\rm cm}^{-2} {\rm s}^{-1} {\mu{\rm m}}^{-1}}$
$\newcommand{\cm}{ {\rm cm}^{-2}}$
$\newcommand{\mm}{ {\rm mm}}$
$\newcommand{\mum}{ {\mu{\rm m}}}$
$\newcommand{\kev}{ {\rm keV}}$
$\newcommand{\ev}{ {\rm eV}}$
$\newcommand{\jy}{ {\rm Jy}}$
$\newcommand{\mjy}{ {\rm mJy}}$
$\newcommand{\msun}{ {\rm M_{\odot}}}$
$\newcommand{\lsun}{ {\rm L_{\odot}}}$
$\newcommand{\dg}{^{\circ}}$
$\newcommand{\sqas}{\Box ^{\prime\prime}}$
$\newcommand{\as}{^{\prime\prime}}$
$\newcommand{\am}{^{\prime}}$
$\newcommand{\bdm}{\begin{displaymath}}$
$\newcommand{\edm}{\end{displaymath}}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\bit}{\begin{itemize}}$
$\newcommand{\eit}{\end{itemize}}$
$\newcommand{\ben}{\begin{enumerate}}$
$\newcommand{\een}{\end{enumerate}}$
$\newcommand{\bfi}{\begin{figure}[htb]}$
$\newcommand{\bpfi}{\begin{figure}[p]}$
$\newcommand{\lbol}{\rm L_{Bol}}$
$\newcommand{\llyc}{\rm L_{Lyc}}$
$\newcommand{\lk}{\rm L_K}$
$\newcommand{\av}{\rm A_V}$
$\newcommand{\mv}{\rm M_V}$
$\newcommand{\mb}{\rm M_B}$
$\newcommand{\htwo}{\rm H_2}$
$\newcommand{\fetwo}{\rm[Fe II]}$
$\newcommand{\lir}{\rm L_{IR}}$
$\newcommand{\brg}{\rm Br\gamma}$
$\newcommand{\ha}{\rm H\alpha}$
$\newcommand{\hb}{\rm H\beta}$
$\newcommand{\hel}{\rm He I}$
$\newcommand{\paa}{\rm Pa\alpha}$
$\newcommand{\mhtwo}{M_{\rm H_2}}$
$\newcommand{\htwoline}{\rm H_2 (1-0) S1}$
$\begin{document}$
$\title{In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the James Webb Space Telescope}$
$\author[0000-0002-5666-7782]{T. B\"oker}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6881-0574]{T. L. Beck}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7058-1726]{S. M. Birkmann}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-9262-7155]{G. Giardino}$
$\affiliation{ATG Europe for the European Space Agency, ESTEC, Noordwijk, The Netherlands}$
$\author[0000-0002-4834-369X]{C. Keyes}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-5320-2568]{N. Kumari}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{J. Muzerolle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-7028-5588]{T. Rawle}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6091-7924]{P. Zeidler}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{Y. Abul-Huda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2896-4138]{C. Alves de Oliveira}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0001-7997-1640]{S. Arribas}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0002-7722-6900]{K. Bechtold}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-0883-2226]{R. Bhatawdekar}$
$\affiliation{European Space Agency, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands}$
$\author[0000-0001-8470-7094]{N. Bonaventura}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{A. J. Bunker}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-0450-7306]{A. J. Cameron}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-6719-380X]{S. Carniani}$
$\affiliation{Scuola Normale Superiore, Piazza dei Cavalieri 7, I-56126 Pisa, Italy}$
$\author[0000-0003-3458-2275]{S. Charlot}$
$\affiliation{Sorbonne Universit\'e, UPMC-CNRS, UMR7095, Institut d'Astrophysique de Paris, F-75014 Paris, France}$
$\author[0000-0002-2678-2560]{M. Curti}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0001-9513-1449]{N. Espinoza}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\affiliation{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0001-8895-0606]{P. Ferruit}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0002-8871-3027]{M. Franx}$
$\affiliation{Leiden Observatory, Leiden University, PO Box 9513, 2300RA Leiden, The Netherlands}$
$\author[0000-0002-6780-2441]{P. Jakobsen}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{D. Karakla}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1016-9283]{ M.~López-Caniego}$
$\affiliation{Aurora Technology for the European Space Agency, Villanueva de la Ca\~nada, E-28692 Madrid, Spain.}$
$\affiliation{Universidad Europea de Madrid, 28670, Madrid, Spain.}$
$\author[0000-0002-4034-0080]{N. L\"utzgendorf}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4985-3819]{R. Maiolino}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0003-0192-6887]{E. Manjavacas}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-5788-5258]{A. P. Marston}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author{S. H. Moseley}$
$\affiliation{Quantum Circuits, Inc., New Haven, Connecticut, USA}$
$\author[0000-0002-3471-981X]{P. Ogle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0362-5941]{M. Perna}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0003-2314-3453]{M. Pe\~na-Guerrero}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{N. Pirzkal}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2509-3878]{R. Plesha}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7617-5665]{C. R. Proffitt}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2662-6821]{B. J. Rauscher}$
$\affiliation{NASA Goddard Space Flight Center, Observational Cosmology Laboratory, Greenbelt, USA}$
$\author{H.-W. Rix }$
$\affiliation{Max-Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany}$
$\author[0000-0001-5171-3930]{B. Rodr\'iguez del Pino}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author{Z. Rustamkulov}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0003-2954-7643]{E. Sabbi}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-6050-7645]{D. K. Sing}$
$\affil{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author{M. Sirianni}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{M. te Plate}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7130-2880]{L. \'Ubeda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6570-4776]{G. M. Wahlgren}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{E. Wislowski}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{R. Wu}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4201-7367]{Chris J. Willott}$
$\affil{NRC Herzberg, 5071 West Saanich Rd, Victoria, BC V9E 2E7, Canada}$
$\begin{abstract}$
$The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectrophotometric and wavelength calibration of NIRSpec spectra, and new the 'as measured' sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs.$
$\end{abstract}$
$\keywords{Instrumentation: spectrographs - Space vehicles: instruments}$
$\n\end{document}\end{figure}}\end{figure}}\end{enumerate}}\end{itemize}}\end{equation}}\end{displaymath}}$
$\newcommand{\edm}{\end{displaymath}}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\bit}{\begin{itemize}}$
$\newcommand{\eit}{\end{itemize}}$
$\newcommand{\ben}{\begin{enumerate}}$
$\newcommand{\een}{\end{enumerate}}$
$\newcommand{\bfi}{\begin{figure}[htb]}$
$\newcommand{\bpfi}{\begin{figure}[p]}$
$\newcommand{\lbol}{\rm L_{Bol}}$
$\newcommand{\llyc}{\rm L_{Lyc}}$
$\newcommand{\lk}{\rm L_K}$
$\newcommand{\av}{\rm A_V}$
$\newcommand{\mv}{\rm M_V}$
$\newcommand{\mb}{\rm M_B}$
$\newcommand{\htwo}{\rm H_2}$
$\newcommand{\fetwo}{\rm[Fe II]}$
$\newcommand{\lir}{\rm L_{IR}}$
$\newcommand{\brg}{\rm Br\gamma}$
$\newcommand{\ha}{\rm H\alpha}$
$\newcommand{\hb}{\rm H\beta}$
$\newcommand{\hel}{\rm He I}$
$\newcommand{\paa}{\rm Pa\alpha}$
$\newcommand{\mhtwo}{M_{\rm H_2}}$
$\newcommand{\htwoline}{\rm H_2 (1-0) S1}$
$\begin{document}$
$\title{In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the James Webb Space Telescope}$
$\author[0000-0002-5666-7782]{T. B\"oker}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6881-0574]{T. L. Beck}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7058-1726]{S. M. Birkmann}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-9262-7155]{G. Giardino}$
$\affiliation{ATG Europe for the European Space Agency, ESTEC, Noordwijk, The Netherlands}$
$\author[0000-0002-4834-369X]{C. Keyes}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-5320-2568]{N. Kumari}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{J. Muzerolle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-7028-5588]{T. Rawle}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6091-7924]{P. Zeidler}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{Y. Abul-Huda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2896-4138]{C. Alves de Oliveira}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0001-7997-1640]{S. Arribas}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0002-7722-6900]{K. Bechtold}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-0883-2226]{R. Bhatawdekar}$
$\affiliation{European Space Agency, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands}$
$\author[0000-0001-8470-7094]{N. Bonaventura}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{A. J. Bunker}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-0450-7306]{A. J. Cameron}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-6719-380X]{S. Carniani}$
$\affiliation{Scuola Normale Superiore, Piazza dei Cavalieri 7, I-56126 Pisa, Italy}$
$\author[0000-0003-3458-2275]{S. Charlot}$
$\affiliation{Sorbonne Universit\'e, UPMC-CNRS, UMR7095, Institut d'Astrophysique de Paris, F-75014 Paris, France}$
$\author[0000-0002-2678-2560]{M. Curti}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0001-9513-1449]{N. Espinoza}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\affiliation{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0001-8895-0606]{P. Ferruit}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0002-8871-3027]{M. Franx}$
$\affiliation{Leiden Observatory, Leiden University, PO Box 9513, 2300RA Leiden, The Netherlands}$
$\author[0000-0002-6780-2441]{P. Jakobsen}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{D. Karakla}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1016-9283]{ M.~López-Caniego}$
$\affiliation{Aurora Technology for the European Space Agency, Villanueva de la Ca\~nada, E-28692 Madrid, Spain.}$
$\affiliation{Universidad Europea de Madrid, 28670, Madrid, Spain.}$
$\author[0000-0002-4034-0080]{N. L\"utzgendorf}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4985-3819]{R. Maiolino}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0003-0192-6887]{E. Manjavacas}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-5788-5258]{A. P. Marston}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author{S. H. Moseley}$
$\affiliation{Quantum Circuits, Inc., New Haven, Connecticut, USA}$
$\author[0000-0002-3471-981X]{P. Ogle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0362-5941]{M. Perna}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0003-2314-3453]{M. Pe\~na-Guerrero}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{N. Pirzkal}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2509-3878]{R. Plesha}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7617-5665]{C. R. Proffitt}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2662-6821]{B. J. Rauscher}$
$\affiliation{NASA Goddard Space Flight Center, Observational Cosmology Laboratory, Greenbelt, USA}$
$\author{H.-W. Rix }$
$\affiliation{Max-Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany}$
$\author[0000-0001-5171-3930]{B. Rodr\'iguez del Pino}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author{Z. Rustamkulov}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0003-2954-7643]{E. Sabbi}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-6050-7645]{D. K. Sing}$
$\affil{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author{M. Sirianni}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{M. te Plate}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7130-2880]{L. \'Ubeda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6570-4776]{G. M. Wahlgren}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{E. Wislowski}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{R. Wu}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4201-7367]{Chris J. Willott}$
$\affil{NRC Herzberg, 5071 West Saanich Rd, Victoria, BC V9E 2E7, Canada}$
$\begin{abstract}$
$The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectrophotometric and wavelength calibration of NIRSpec spectra, and new the 'as measured' sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs.$
$\end{abstract}$
$\keywords{Instrumentation: spectrographs - Space vehicles: instruments}$
$\n\end{document}\end{figure}}\end{figure}}\end{enumerate}}\end{itemize}}\end{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\bit}{\begin{itemize}}$
$\newcommand{\eit}{\end{itemize}}$
$\newcommand{\ben}{\begin{enumerate}}$
$\newcommand{\een}{\end{enumerate}}$
$\newcommand{\bfi}{\begin{figure}[htb]}$
$\newcommand{\bpfi}{\begin{figure}[p]}$
$\newcommand{\lbol}{\rm L_{Bol}}$
$\newcommand{\llyc}{\rm L_{Lyc}}$
$\newcommand{\lk}{\rm L_K}$
$\newcommand{\av}{\rm A_V}$
$\newcommand{\mv}{\rm M_V}$
$\newcommand{\mb}{\rm M_B}$
$\newcommand{\htwo}{\rm H_2}$
$\newcommand{\fetwo}{\rm[Fe II]}$
$\newcommand{\lir}{\rm L_{IR}}$
$\newcommand{\brg}{\rm Br\gamma}$
$\newcommand{\ha}{\rm H\alpha}$
$\newcommand{\hb}{\rm H\beta}$
$\newcommand{\hel}{\rm He I}$
$\newcommand{\paa}{\rm Pa\alpha}$
$\newcommand{\mhtwo}{M_{\rm H_2}}$
$\newcommand{\htwoline}{\rm H_2 (1-0) S1}$
$\begin{document}$
$\title{In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the James Webb Space Telescope}$
$\author[0000-0002-5666-7782]{T. B\"oker}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6881-0574]{T. L. Beck}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7058-1726]{S. M. Birkmann}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-9262-7155]{G. Giardino}$
$\affiliation{ATG Europe for the European Space Agency, ESTEC, Noordwijk, The Netherlands}$
$\author[0000-0002-4834-369X]{C. Keyes}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-5320-2568]{N. Kumari}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{J. Muzerolle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-7028-5588]{T. Rawle}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6091-7924]{P. Zeidler}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{Y. Abul-Huda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2896-4138]{C. Alves de Oliveira}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0001-7997-1640]{S. Arribas}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0002-7722-6900]{K. Bechtold}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-0883-2226]{R. Bhatawdekar}$
$\affiliation{European Space Agency, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands}$
$\author[0000-0001-8470-7094]{N. Bonaventura}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{A. J. Bunker}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-0450-7306]{A. J. Cameron}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-6719-380X]{S. Carniani}$
$\affiliation{Scuola Normale Superiore, Piazza dei Cavalieri 7, I-56126 Pisa, Italy}$
$\author[0000-0003-3458-2275]{S. Charlot}$
$\affiliation{Sorbonne Universit\'e, UPMC-CNRS, UMR7095, Institut d'Astrophysique de Paris, F-75014 Paris, France}$
$\author[0000-0002-2678-2560]{M. Curti}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0001-9513-1449]{N. Espinoza}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\affiliation{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0001-8895-0606]{P. Ferruit}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0002-8871-3027]{M. Franx}$
$\affiliation{Leiden Observatory, Leiden University, PO Box 9513, 2300RA Leiden, The Netherlands}$
$\author[0000-0002-6780-2441]{P. Jakobsen}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{D. Karakla}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1016-9283]{ M.~López-Caniego}$
$\affiliation{Aurora Technology for the European Space Agency, Villanueva de la Ca\~nada, E-28692 Madrid, Spain.}$
$\affiliation{Universidad Europea de Madrid, 28670, Madrid, Spain.}$
$\author[0000-0002-4034-0080]{N. L\"utzgendorf}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4985-3819]{R. Maiolino}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0003-0192-6887]{E. Manjavacas}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-5788-5258]{A. P. Marston}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author{S. H. Moseley}$
$\affiliation{Quantum Circuits, Inc., New Haven, Connecticut, USA}$
$\author[0000-0002-3471-981X]{P. Ogle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0362-5941]{M. Perna}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0003-2314-3453]{M. Pe\~na-Guerrero}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{N. Pirzkal}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2509-3878]{R. Plesha}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7617-5665]{C. R. Proffitt}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2662-6821]{B. J. Rauscher}$
$\affiliation{NASA Goddard Space Flight Center, Observational Cosmology Laboratory, Greenbelt, USA}$
$\author{H.-W. Rix }$
$\affiliation{Max-Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany}$
$\author[0000-0001-5171-3930]{B. Rodr\'iguez del Pino}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author{Z. Rustamkulov}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0003-2954-7643]{E. Sabbi}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-6050-7645]{D. K. Sing}$
$\affil{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author{M. Sirianni}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{M. te Plate}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7130-2880]{L. \'Ubeda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6570-4776]{G. M. Wahlgren}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{E. Wislowski}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{R. Wu}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4201-7367]{Chris J. Willott}$
$\affil{NRC Herzberg, 5071 West Saanich Rd, Victoria, BC V9E 2E7, Canada}$
$\begin{abstract}$
$The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectrophotometric and wavelength calibration of NIRSpec spectra, and new the 'as measured' sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs.$
$\end{abstract}$
$\keywords{Instrumentation: spectrographs - Space vehicles: instruments}$
$\n\end{document}\end{figure}}\end{figure}}\end{enumerate}}\end{itemize}}$
$\newcommand{\eit}{\end{itemize}}$
$\newcommand{\ben}{\begin{enumerate}}$
$\newcommand{\een}{\end{enumerate}}$
$\newcommand{\bfi}{\begin{figure}[htb]}$
$\newcommand{\bpfi}{\begin{figure}[p]}$
$\newcommand{\lbol}{\rm L_{Bol}}$
$\newcommand{\llyc}{\rm L_{Lyc}}$
$\newcommand{\lk}{\rm L_K}$
$\newcommand{\av}{\rm A_V}$
$\newcommand{\mv}{\rm M_V}$
$\newcommand{\mb}{\rm M_B}$
$\newcommand{\htwo}{\rm H_2}$
$\newcommand{\fetwo}{\rm[Fe II]}$
$\newcommand{\lir}{\rm L_{IR}}$
$\newcommand{\brg}{\rm Br\gamma}$
$\newcommand{\ha}{\rm H\alpha}$
$\newcommand{\hb}{\rm H\beta}$
$\newcommand{\hel}{\rm He I}$
$\newcommand{\paa}{\rm Pa\alpha}$
$\newcommand{\mhtwo}{M_{\rm H_2}}$
$\newcommand{\htwoline}{\rm H_2 (1-0) S1}$
$\begin{document}$
$\title{In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the James Webb Space Telescope}$
$\author[0000-0002-5666-7782]{T. B\"oker}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6881-0574]{T. L. Beck}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7058-1726]{S. M. Birkmann}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-9262-7155]{G. Giardino}$
$\affiliation{ATG Europe for the European Space Agency, ESTEC, Noordwijk, The Netherlands}$
$\author[0000-0002-4834-369X]{C. Keyes}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-5320-2568]{N. Kumari}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{J. Muzerolle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-7028-5588]{T. Rawle}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6091-7924]{P. Zeidler}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{Y. Abul-Huda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2896-4138]{C. Alves de Oliveira}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0001-7997-1640]{S. Arribas}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0002-7722-6900]{K. Bechtold}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-0883-2226]{R. Bhatawdekar}$
$\affiliation{European Space Agency, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands}$
$\author[0000-0001-8470-7094]{N. Bonaventura}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{A. J. Bunker}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-0450-7306]{A. J. Cameron}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-6719-380X]{S. Carniani}$
$\affiliation{Scuola Normale Superiore, Piazza dei Cavalieri 7, I-56126 Pisa, Italy}$
$\author[0000-0003-3458-2275]{S. Charlot}$
$\affiliation{Sorbonne Universit\'e, UPMC-CNRS, UMR7095, Institut d'Astrophysique de Paris, F-75014 Paris, France}$
$\author[0000-0002-2678-2560]{M. Curti}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0001-9513-1449]{N. Espinoza}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\affiliation{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0001-8895-0606]{P. Ferruit}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0002-8871-3027]{M. Franx}$
$\affiliation{Leiden Observatory, Leiden University, PO Box 9513, 2300RA Leiden, The Netherlands}$
$\author[0000-0002-6780-2441]{P. Jakobsen}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{D. Karakla}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1016-9283]{ M.~López-Caniego}$
$\affiliation{Aurora Technology for the European Space Agency, Villanueva de la Ca\~nada, E-28692 Madrid, Spain.}$
$\affiliation{Universidad Europea de Madrid, 28670, Madrid, Spain.}$
$\author[0000-0002-4034-0080]{N. L\"utzgendorf}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4985-3819]{R. Maiolino}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0003-0192-6887]{E. Manjavacas}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-5788-5258]{A. P. Marston}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author{S. H. Moseley}$
$\affiliation{Quantum Circuits, Inc., New Haven, Connecticut, USA}$
$\author[0000-0002-3471-981X]{P. Ogle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0362-5941]{M. Perna}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0003-2314-3453]{M. Pe\~na-Guerrero}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{N. Pirzkal}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2509-3878]{R. Plesha}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7617-5665]{C. R. Proffitt}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2662-6821]{B. J. Rauscher}$
$\affiliation{NASA Goddard Space Flight Center, Observational Cosmology Laboratory, Greenbelt, USA}$
$\author{H.-W. Rix }$
$\affiliation{Max-Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany}$
$\author[0000-0001-5171-3930]{B. Rodr\'iguez del Pino}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author{Z. Rustamkulov}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0003-2954-7643]{E. Sabbi}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-6050-7645]{D. K. Sing}$
$\affil{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author{M. Sirianni}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{M. te Plate}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7130-2880]{L. \'Ubeda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6570-4776]{G. M. Wahlgren}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{E. Wislowski}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{R. Wu}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4201-7367]{Chris J. Willott}$
$\affil{NRC Herzberg, 5071 West Saanich Rd, Victoria, BC V9E 2E7, Canada}$
$\begin{abstract}$
$The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectrophotometric and wavelength calibration of NIRSpec spectra, and new the 'as measured' sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs.$
$\end{abstract}$
$\keywords{Instrumentation: spectrographs - Space vehicles: instruments}$
$\n\end{document}\end{figure}}\end{figure}}\end{enumerate}}$
$\newcommand{\een}{\end{enumerate}}$
$\newcommand{\bfi}{\begin{figure}[htb]}$
$\newcommand{\bpfi}{\begin{figure}[p]}$
$\newcommand{\lbol}{\rm L_{Bol}}$
$\newcommand{\llyc}{\rm L_{Lyc}}$
$\newcommand{\lk}{\rm L_K}$
$\newcommand{\av}{\rm A_V}$
$\newcommand{\mv}{\rm M_V}$
$\newcommand{\mb}{\rm M_B}$
$\newcommand{\htwo}{\rm H_2}$
$\newcommand{\fetwo}{\rm[Fe II]}$
$\newcommand{\lir}{\rm L_{IR}}$
$\newcommand{\brg}{\rm Br\gamma}$
$\newcommand{\ha}{\rm H\alpha}$
$\newcommand{\hb}{\rm H\beta}$
$\newcommand{\hel}{\rm He I}$
$\newcommand{\paa}{\rm Pa\alpha}$
$\newcommand{\mhtwo}{M_{\rm H_2}}$
$\newcommand{\htwoline}{\rm H_2 (1-0) S1}$
$\begin{document}$
$\title{In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the James Webb Space Telescope}$
$\author[0000-0002-5666-7782]{T. B\"oker}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6881-0574]{T. L. Beck}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7058-1726]{S. M. Birkmann}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-9262-7155]{G. Giardino}$
$\affiliation{ATG Europe for the European Space Agency, ESTEC, Noordwijk, The Netherlands}$
$\author[0000-0002-4834-369X]{C. Keyes}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-5320-2568]{N. Kumari}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{J. Muzerolle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-7028-5588]{T. Rawle}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6091-7924]{P. Zeidler}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{Y. Abul-Huda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2896-4138]{C. Alves de Oliveira}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0001-7997-1640]{S. Arribas}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0002-7722-6900]{K. Bechtold}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-0883-2226]{R. Bhatawdekar}$
$\affiliation{European Space Agency, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands}$
$\author[0000-0001-8470-7094]{N. Bonaventura}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{A. J. Bunker}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-0450-7306]{A. J. Cameron}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-6719-380X]{S. Carniani}$
$\affiliation{Scuola Normale Superiore, Piazza dei Cavalieri 7, I-56126 Pisa, Italy}$
$\author[0000-0003-3458-2275]{S. Charlot}$
$\affiliation{Sorbonne Universit\'e, UPMC-CNRS, UMR7095, Institut d'Astrophysique de Paris, F-75014 Paris, France}$
$\author[0000-0002-2678-2560]{M. Curti}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0001-9513-1449]{N. Espinoza}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\affiliation{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0001-8895-0606]{P. Ferruit}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0002-8871-3027]{M. Franx}$
$\affiliation{Leiden Observatory, Leiden University, PO Box 9513, 2300RA Leiden, The Netherlands}$
$\author[0000-0002-6780-2441]{P. Jakobsen}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{D. Karakla}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1016-9283]{ M.~López-Caniego}$
$\affiliation{Aurora Technology for the European Space Agency, Villanueva de la Ca\~nada, E-28692 Madrid, Spain.}$
$\affiliation{Universidad Europea de Madrid, 28670, Madrid, Spain.}$
$\author[0000-0002-4034-0080]{N. L\"utzgendorf}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4985-3819]{R. Maiolino}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0003-0192-6887]{E. Manjavacas}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-5788-5258]{A. P. Marston}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author{S. H. Moseley}$
$\affiliation{Quantum Circuits, Inc., New Haven, Connecticut, USA}$
$\author[0000-0002-3471-981X]{P. Ogle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0362-5941]{M. Perna}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0003-2314-3453]{M. Pe\~na-Guerrero}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{N. Pirzkal}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2509-3878]{R. Plesha}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7617-5665]{C. R. Proffitt}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2662-6821]{B. J. Rauscher}$
$\affiliation{NASA Goddard Space Flight Center, Observational Cosmology Laboratory, Greenbelt, USA}$
$\author{H.-W. Rix }$
$\affiliation{Max-Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany}$
$\author[0000-0001-5171-3930]{B. Rodr\'iguez del Pino}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author{Z. Rustamkulov}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0003-2954-7643]{E. Sabbi}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-6050-7645]{D. K. Sing}$
$\affil{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author{M. Sirianni}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{M. te Plate}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7130-2880]{L. \'Ubeda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6570-4776]{G. M. Wahlgren}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{E. Wislowski}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{R. Wu}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4201-7367]{Chris J. Willott}$
$\affil{NRC Herzberg, 5071 West Saanich Rd, Victoria, BC V9E 2E7, Canada}$
$\begin{abstract}$
$The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectrophotometric and wavelength calibration of NIRSpec spectra, and new the 'as measured' sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs.$
$\end{abstract}$
$\keywords{Instrumentation: spectrographs - Space vehicles: instruments}$
$\n\end{document}\end{figure}}\end{figure}}$
$\newcommand{\bpfi}{\begin{figure}[p]}$
$\newcommand{\lbol}{\rm L_{Bol}}$
$\newcommand{\llyc}{\rm L_{Lyc}}$
$\newcommand{\lk}{\rm L_K}$
$\newcommand{\av}{\rm A_V}$
$\newcommand{\mv}{\rm M_V}$
$\newcommand{\mb}{\rm M_B}$
$\newcommand{\htwo}{\rm H_2}$
$\newcommand{\fetwo}{\rm[Fe II]}$
$\newcommand{\lir}{\rm L_{IR}}$
$\newcommand{\brg}{\rm Br\gamma}$
$\newcommand{\ha}{\rm H\alpha}$
$\newcommand{\hb}{\rm H\beta}$
$\newcommand{\hel}{\rm He I}$
$\newcommand{\paa}{\rm Pa\alpha}$
$\newcommand{\mhtwo}{M_{\rm H_2}}$
$\newcommand{\htwoline}{\rm H_2 (1-0) S1}$
$\begin{document}$
$\title{In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the James Webb Space Telescope}$
$\author[0000-0002-5666-7782]{T. B\"oker}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6881-0574]{T. L. Beck}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7058-1726]{S. M. Birkmann}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-9262-7155]{G. Giardino}$
$\affiliation{ATG Europe for the European Space Agency, ESTEC, Noordwijk, The Netherlands}$
$\author[0000-0002-4834-369X]{C. Keyes}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-5320-2568]{N. Kumari}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{J. Muzerolle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-7028-5588]{T. Rawle}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6091-7924]{P. Zeidler}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{Y. Abul-Huda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2896-4138]{C. Alves de Oliveira}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0001-7997-1640]{S. Arribas}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0002-7722-6900]{K. Bechtold}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-0883-2226]{R. Bhatawdekar}$
$\affiliation{European Space Agency, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands}$
$\author[0000-0001-8470-7094]{N. Bonaventura}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{A. J. Bunker}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-0450-7306]{A. J. Cameron}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-6719-380X]{S. Carniani}$
$\affiliation{Scuola Normale Superiore, Piazza dei Cavalieri 7, I-56126 Pisa, Italy}$
$\author[0000-0003-3458-2275]{S. Charlot}$
$\affiliation{Sorbonne Universit\'e, UPMC-CNRS, UMR7095, Institut d'Astrophysique de Paris, F-75014 Paris, France}$
$\author[0000-0002-2678-2560]{M. Curti}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0001-9513-1449]{N. Espinoza}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\affiliation{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0001-8895-0606]{P. Ferruit}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0002-8871-3027]{M. Franx}$
$\affiliation{Leiden Observatory, Leiden University, PO Box 9513, 2300RA Leiden, The Netherlands}$
$\author[0000-0002-6780-2441]{P. Jakobsen}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{D. Karakla}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1016-9283]{ M.~López-Caniego}$
$\affiliation{Aurora Technology for the European Space Agency, Villanueva de la Ca\~nada, E-28692 Madrid, Spain.}$
$\affiliation{Universidad Europea de Madrid, 28670, Madrid, Spain.}$
$\author[0000-0002-4034-0080]{N. L\"utzgendorf}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4985-3819]{R. Maiolino}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0003-0192-6887]{E. Manjavacas}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-5788-5258]{A. P. Marston}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author{S. H. Moseley}$
$\affiliation{Quantum Circuits, Inc., New Haven, Connecticut, USA}$
$\author[0000-0002-3471-981X]{P. Ogle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0362-5941]{M. Perna}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0003-2314-3453]{M. Pe\~na-Guerrero}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{N. Pirzkal}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2509-3878]{R. Plesha}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7617-5665]{C. R. Proffitt}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2662-6821]{B. J. Rauscher}$
$\affiliation{NASA Goddard Space Flight Center, Observational Cosmology Laboratory, Greenbelt, USA}$
$\author{H.-W. Rix }$
$\affiliation{Max-Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany}$
$\author[0000-0001-5171-3930]{B. Rodr\'iguez del Pino}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author{Z. Rustamkulov}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0003-2954-7643]{E. Sabbi}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-6050-7645]{D. K. Sing}$
$\affil{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author{M. Sirianni}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{M. te Plate}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7130-2880]{L. \'Ubeda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6570-4776]{G. M. Wahlgren}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{E. Wislowski}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{R. Wu}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4201-7367]{Chris J. Willott}$
$\affil{NRC Herzberg, 5071 West Saanich Rd, Victoria, BC V9E 2E7, Canada}$
$\begin{abstract}$
$The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectrophotometric and wavelength calibration of NIRSpec spectra, and new the 'as measured' sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs.$
$\end{abstract}$
$\keywords{Instrumentation: spectrographs - Space vehicles: instruments}$
$\n\end{document}\end{figure}}$
$\newcommand{\lbol}{\rm L_{Bol}}$
$\newcommand{\llyc}{\rm L_{Lyc}}$
$\newcommand{\lk}{\rm L_K}$
$\newcommand{\av}{\rm A_V}$
$\newcommand{\mv}{\rm M_V}$
$\newcommand{\mb}{\rm M_B}$
$\newcommand{\htwo}{\rm H_2}$
$\newcommand{\fetwo}{\rm[Fe II]}$
$\newcommand{\lir}{\rm L_{IR}}$
$\newcommand{\brg}{\rm Br\gamma}$
$\newcommand{\ha}{\rm H\alpha}$
$\newcommand{\hb}{\rm H\beta}$
$\newcommand{\hel}{\rm He I}$
$\newcommand{\paa}{\rm Pa\alpha}$
$\newcommand{\mhtwo}{M_{\rm H_2}}$
$\newcommand{\htwoline}{\rm H_2 (1-0) S1}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\lta$}{\lesssim}$
$\newcommand{$\gta$}{\gtrsim}$
$\newcommand{$\coone$}{\rm ^{12}CO(1-0)}$
$\newcommand{$\cotwo$}{\rm ^{12}CO(2-1)}$
$\newcommand{$\one$}{\rm (1-0)}$
$\newcommand{$\two$}{\rm (2-1)}$
$\newcommand{$\ion$e}{I_{10}}$
$\newcommand{$\itwo$}{I_{21}}$
$\newcommand{$\nhico$}{X_{CO}}$
$\newcommand{$\CO$}{^{12}CO }$
$\newcommand{$\CO$e}{CO(1-0)}$
$\newcommand{$\CO$z}{CO(2-1) }$
$\newcommand{$\solm$}{M_{\odot}}$
$\newcommand{$\solm$s}{ {\rm M_{\odot}/pc^2}}$
$\newcommand{$\rsol$}{R_{\odot}}$
$\newcommand{$\rf$}{\par\noindent$\ha$ngindent 15pt  }$
$\newcommand{$\ea$}{et al.}$
$\newcommand{$\kkms$}{ {\rm K} {\rm km} {\rm s}^{-1}}$
$\newcommand{$\kms$}{ {\rm km} {\rm s}^{-1}}$
$\newcommand{$\gyr$}{ {\rm Gyr}}$
$\newcommand{$\myr$}{ {\rm Myr}}$
$\newcommand{$\yr$}{ {\rm yr}}$
$\newcommand{$\s$}{ {\rm s}}$
$\newcommand{$\pc$}{ {\rm pc}}$
$\newcommand{$\kpc$}{ {\rm kpc}}$
$\newcommand{$\mpc$}{ {\rm Mpc}}$
$\newcommand{$\m$}{ {\rm m}}$
$\newcommand{$\km$}{ {\rm km}}$
$\newcommand{$\ergs$}{ {\rm erg} {\rm s}^{-1}}$
$\newcommand{$\ergs$mic}{ {\rm erg} {\rm cm}^{-2} {\rm s}^{-1} {$\m$u{\rm m}}^{-1}}$
$\newcommand{$\cm$}{ {\rm cm}^{-2}}$
$\newcommand{$\m$m}{ {\rm mm}}$
$\newcommand{$\m$um}{ {$\m$u{\rm m}}}$
$\newcommand{$\kev$}{ {\rm keV}}$
$\newcommand{$\ev$}{ {\rm eV}}$
$\newcommand{$\jy$}{ {\rm Jy}}$
$\newcommand{$\m$jy}{ {\rm mJy}}$
$\newcommand{$\m$sun}{ {\rm M_{\odot}}}$
$\newcommand{$\lsun$}{ {\rm L_{\odot}}}$
$\newcommand{$\dg$}{^{\circ}}$
$\newcommand{$\s$qas}{\Box ^{\prime\prime}}$
$\newcommand{$\as$}{^{\prime\prime}}$
$\newcommand{$\am$}{^{\prime}}$
$\newcommand{\bdm}{\begin{displaymath}}$
$\newcommand{$\edm$}{\end{displaymath}}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{$\eeq$}{\end{equation}}$
$\newcommand{\bit}{\begin{itemize}}$
$\newcommand{$\eit$}{\end{itemize}}$
$\newcommand{\ben}{\begin{enumerate}}$
$\newcommand{$\een$}{\end{enumerate}}$
$\newcommand{\bfi}{\begin{figure}[htb]}$
$\newcommand{\bpfi}{\begin{figure}[p]}$
$\newcommand{$\lbol$}{\rm L_{Bol}}$
$\newcommand{$\llyc$}{\rm L_{Lyc}}$
$\newcommand{$\lk$}{\rm L_K}$
$\newcommand{$\av$}{\rm A_V}$
$\newcommand{$\m$v}{\rm M_V}$
$\newcommand{$\m$b}{\rm M_B}$
$\newcommand{$\htwo$}{\rm H_2}$
$\newcommand{$\fetwo$}{\rm[Fe II]}$
$\newcommand{$\lir$}{\rm L_{IR}}$
$\newcommand{$\brg$}{\rm Br\gamma}$
$\newcommand{$\ha$}{\rm H\alpha}$
$\newcommand{$\hb$}{\rm H\beta}$
$\newcommand{$\hel$}{\rm He I}$
$\newcommand{$\paa$}{\rm Pa\alpha}$
$\newcommand{$\m$htwo}{M_{\rm H_2}}$
$\newcommand{$\htwo$line}{\rm H_2 (1-0) S1}$
$\begin{document}$
$\title{In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the James Webb Space Telescope}$
$\author[0000-0002-5666-7782]{T. B\"oker}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6881-0574]{T. L. Beck}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7058-1726]{S. M. Birkmann}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-9262-7155]{G. Giardino}$
$\affiliation{ATG Europe for the European Space Agency, ESTEC, Noordwijk, The Netherlands}$
$\author[0000-0002-4834-369X]{C. Keyes}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-5320-2568]{N. Kumari}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{J. Muzerolle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-7028-5588]{T. Rawle}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6091-7924]{P. Zeidler}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{Y. Abul-Huda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2896-4138]{C. Alves de Oliveira}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0001-7997-1640]{S. Arribas}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0002-7722-6900]{K. Bechtold}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-0883-2226]{R. Bhatawdekar}$
$\affiliation{European Space Agency, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands}$
$\author[0000-0001-8470-7094]{N. Bonaventura}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{A. J. Bunker}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-0450-7306]{A. J. Cameron}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-6719-380X]{S. Carniani}$
$\affiliation{Scuola Normale Superiore, Piazza dei Cavalieri 7, I-56126 Pisa, Italy}$
$\author[0000-0003-3458-2275]{S. Charlot}$
$\affiliation{Sorbonne Universit\'e, UPMC-CNRS, UMR7095, Institut d'Astrophysique de Paris, F-75014 Paris, France}$
$\author[0000-0002-2678-2560]{M. Curti}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0001-9513-1449]{N. Espinoza}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\affiliation{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0001-8895-0606]{P. Ferruit}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0002-8871-3027]{M. Franx}$
$\affiliation{Leiden Observatory, Leiden University, PO Box 9513, 2300RA Leiden, The Netherlands}$
$\author[0000-0002-6780-2441]{P. Jakobsen}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{D. Karakla}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1016-9283]{ M.~López-Caniego}$
$\affiliation{Aurora Technology for the European Space Agency, Villanueva de la Ca\~nada, E-28692 Madrid, Spain.}$
$\affiliation{Universidad Europea de Madrid, 28670, Madrid, Spain.}$
$\author[0000-0002-4034-0080]{N. L\"utzgendorf}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4985-3819]{R. Maiolino}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0003-0192-6887]{E. Manjavacas}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-5788-5258]{A. P. Marston}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author{S. H. Moseley}$
$\affiliation{Quantum Circuits, Inc., New Haven, Connecticut, USA}$
$\author[0000-0002-3471-981X]{P. Ogle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0362-5941]{M. Perna}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0003-2314-3453]{M. Pe\~na-Guerrero}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{N. Pirzkal}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2509-3878]{R. Plesha}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7617-5665]{C. R. Proffitt}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2662-6821]{B. J. Rauscher}$
$\affiliation{NASA Goddard Space Flight Center, Observational Cosmology Laboratory, Greenbelt, USA}$
$\author{H.-W. Rix }$
$\affiliation{Max-Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany}$
$\author[0000-0001-5171-3930]{B. Rodr\'iguez del Pino}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author{Z. Rustamkulov}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0003-2954-7643]{E. Sabbi}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-6050-7645]{D. K. Sing}$
$\affil{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author{M. Sirianni}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{M. te Plate}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7130-2880]{L. \'Ubeda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6570-4776]{G. M. Wahlgren}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{E. Wislowski}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{R. Wu}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4201-7367]{Chris J. Willott}$
$\affil{NRC Herzberg, 5071 West Saanich Rd, Victoria, BC V9E 2E7, Canada}$
$\begin{abstract}$
$The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectrophotometric and wavelength calibration of NIRSpec spectra, and new the 'as measured' sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs.$
$\end{abstract}$
$\keywords{Instrumentation: spectrographs - Space vehicles: instruments}$
$\n\end{document}\end{figure}}\end{figure}}\end{enumerate}}\end{itemize}}\end{equation}}\end{displaymath}}$
$\newcommand{$\edm$}{\end{displaymath}}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{$\eeq$}{\end{equation}}$
$\newcommand{\bit}{\begin{itemize}}$
$\newcommand{$\eit$}{\end{itemize}}$
$\newcommand{\ben}{\begin{enumerate}}$
$\newcommand{$\een$}{\end{enumerate}}$
$\newcommand{\bfi}{\begin{figure}[htb]}$
$\newcommand{\bpfi}{\begin{figure}[p]}$
$\newcommand{$\lbol$}{\rm L_{Bol}}$
$\newcommand{$\llyc$}{\rm L_{Lyc}}$
$\newcommand{$\lk$}{\rm L_K}$
$\newcommand{$\av$}{\rm A_V}$
$\newcommand{$\m$v}{\rm M_V}$
$\newcommand{$\m$b}{\rm M_B}$
$\newcommand{$\htwo$}{\rm H_2}$
$\newcommand{$\fetwo$}{\rm[Fe II]}$
$\newcommand{$\lir$}{\rm L_{IR}}$
$\newcommand{$\brg$}{\rm Br\gamma}$
$\newcommand{$\ha$}{\rm H\alpha}$
$\newcommand{$\hb$}{\rm H\beta}$
$\newcommand{$\hel$}{\rm He I}$
$\newcommand{$\paa$}{\rm Pa\alpha}$
$\newcommand{$\m$htwo}{M_{\rm H_2}}$
$\newcommand{$\htwo$line}{\rm H_2 (1-0) S1}$
$\begin{document}$
$\title{In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the James Webb Space Telescope}$
$\author[0000-0002-5666-7782]{T. B\"oker}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6881-0574]{T. L. Beck}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7058-1726]{S. M. Birkmann}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-9262-7155]{G. Giardino}$
$\affiliation{ATG Europe for the European Space Agency, ESTEC, Noordwijk, The Netherlands}$
$\author[0000-0002-4834-369X]{C. Keyes}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-5320-2568]{N. Kumari}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{J. Muzerolle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-7028-5588]{T. Rawle}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6091-7924]{P. Zeidler}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{Y. Abul-Huda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2896-4138]{C. Alves de Oliveira}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0001-7997-1640]{S. Arribas}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0002-7722-6900]{K. Bechtold}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-0883-2226]{R. Bhatawdekar}$
$\affiliation{European Space Agency, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands}$
$\author[0000-0001-8470-7094]{N. Bonaventura}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{A. J. Bunker}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-0450-7306]{A. J. Cameron}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-6719-380X]{S. Carniani}$
$\affiliation{Scuola Normale Superiore, Piazza dei Cavalieri 7, I-56126 Pisa, Italy}$
$\author[0000-0003-3458-2275]{S. Charlot}$
$\affiliation{Sorbonne Universit\'e, UPMC-CNRS, UMR7095, Institut d'Astrophysique de Paris, F-75014 Paris, France}$
$\author[0000-0002-2678-2560]{M. Curti}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0001-9513-1449]{N. Espinoza}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\affiliation{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0001-8895-0606]{P. Ferruit}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0002-8871-3027]{M. Franx}$
$\affiliation{Leiden Observatory, Leiden University, PO Box 9513, 2300RA Leiden, The Netherlands}$
$\author[0000-0002-6780-2441]{P. Jakobsen}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{D. Karakla}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1016-9283]{ M.~López-Caniego}$
$\affiliation{Aurora Technology for the European Space Agency, Villanueva de la Ca\~nada, E-28692 Madrid, Spain.}$
$\affiliation{Universidad Europea de Madrid, 28670, Madrid, Spain.}$
$\author[0000-0002-4034-0080]{N. L\"utzgendorf}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4985-3819]{R. Maiolino}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0003-0192-6887]{E. Manjavacas}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-5788-5258]{A. P. Marston}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author{S. H. Moseley}$
$\affiliation{Quantum Circuits, Inc., New Haven, Connecticut, USA}$
$\author[0000-0002-3471-981X]{P. Ogle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0362-5941]{M. Perna}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0003-2314-3453]{M. Pe\~na-Guerrero}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{N. Pirzkal}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2509-3878]{R. Plesha}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7617-5665]{C. R. Proffitt}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2662-6821]{B. J. Rauscher}$
$\affiliation{NASA Goddard Space Flight Center, Observational Cosmology Laboratory, Greenbelt, USA}$
$\author{H.-W. Rix }$
$\affiliation{Max-Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany}$
$\author[0000-0001-5171-3930]{B. Rodr\'iguez del Pino}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author{Z. Rustamkulov}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0003-2954-7643]{E. Sabbi}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-6050-7645]{D. K. Sing}$
$\affil{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author{M. Sirianni}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{M. te Plate}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7130-2880]{L. \'Ubeda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6570-4776]{G. M. Wahlgren}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{E. Wislowski}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{R. Wu}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4201-7367]{Chris J. Willott}$
$\affil{NRC Herzberg, 5071 West Saanich Rd, Victoria, BC V9E 2E7, Canada}$
$\begin{abstract}$
$The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectrophotometric and wavelength calibration of NIRSpec spectra, and new the 'as measured' sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs.$
$\end{abstract}$
$\keywords{Instrumentation: spectrographs - Space vehicles: instruments}$
$\n\end{document}\end{figure}}\end{figure}}\end{enumerate}}\end{itemize}}\end{equation}}$
$\newcommand{$\eeq$}{\end{equation}}$
$\newcommand{\bit}{\begin{itemize}}$
$\newcommand{$\eit$}{\end{itemize}}$
$\newcommand{\ben}{\begin{enumerate}}$
$\newcommand{$\een$}{\end{enumerate}}$
$\newcommand{\bfi}{\begin{figure}[htb]}$
$\newcommand{\bpfi}{\begin{figure}[p]}$
$\newcommand{$\lbol$}{\rm L_{Bol}}$
$\newcommand{$\llyc$}{\rm L_{Lyc}}$
$\newcommand{$\lk$}{\rm L_K}$
$\newcommand{$\av$}{\rm A_V}$
$\newcommand{$\m$v}{\rm M_V}$
$\newcommand{$\m$b}{\rm M_B}$
$\newcommand{$\htwo$}{\rm H_2}$
$\newcommand{$\fetwo$}{\rm[Fe II]}$
$\newcommand{$\lir$}{\rm L_{IR}}$
$\newcommand{$\brg$}{\rm Br\gamma}$
$\newcommand{$\ha$}{\rm H\alpha}$
$\newcommand{$\hb$}{\rm H\beta}$
$\newcommand{$\hel$}{\rm He I}$
$\newcommand{$\paa$}{\rm Pa\alpha}$
$\newcommand{$\m$htwo}{M_{\rm H_2}}$
$\newcommand{$\htwo$line}{\rm H_2 (1-0) S1}$
$\begin{document}$
$\title{In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the James Webb Space Telescope}$
$\author[0000-0002-5666-7782]{T. B\"oker}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6881-0574]{T. L. Beck}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7058-1726]{S. M. Birkmann}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-9262-7155]{G. Giardino}$
$\affiliation{ATG Europe for the European Space Agency, ESTEC, Noordwijk, The Netherlands}$
$\author[0000-0002-4834-369X]{C. Keyes}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-5320-2568]{N. Kumari}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{J. Muzerolle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-7028-5588]{T. Rawle}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6091-7924]{P. Zeidler}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{Y. Abul-Huda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2896-4138]{C. Alves de Oliveira}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0001-7997-1640]{S. Arribas}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0002-7722-6900]{K. Bechtold}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-0883-2226]{R. Bhatawdekar}$
$\affiliation{European Space Agency, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands}$
$\author[0000-0001-8470-7094]{N. Bonaventura}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{A. J. Bunker}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-0450-7306]{A. J. Cameron}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-6719-380X]{S. Carniani}$
$\affiliation{Scuola Normale Superiore, Piazza dei Cavalieri 7, I-56126 Pisa, Italy}$
$\author[0000-0003-3458-2275]{S. Charlot}$
$\affiliation{Sorbonne Universit\'e, UPMC-CNRS, UMR7095, Institut d'Astrophysique de Paris, F-75014 Paris, France}$
$\author[0000-0002-2678-2560]{M. Curti}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0001-9513-1449]{N. Espinoza}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\affiliation{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0001-8895-0606]{P. Ferruit}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0002-8871-3027]{M. Franx}$
$\affiliation{Leiden Observatory, Leiden University, PO Box 9513, 2300RA Leiden, The Netherlands}$
$\author[0000-0002-6780-2441]{P. Jakobsen}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{D. Karakla}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1016-9283]{ M.~López-Caniego}$
$\affiliation{Aurora Technology for the European Space Agency, Villanueva de la Ca\~nada, E-28692 Madrid, Spain.}$
$\affiliation{Universidad Europea de Madrid, 28670, Madrid, Spain.}$
$\author[0000-0002-4034-0080]{N. L\"utzgendorf}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4985-3819]{R. Maiolino}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0003-0192-6887]{E. Manjavacas}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-5788-5258]{A. P. Marston}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author{S. H. Moseley}$
$\affiliation{Quantum Circuits, Inc., New Haven, Connecticut, USA}$
$\author[0000-0002-3471-981X]{P. Ogle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0362-5941]{M. Perna}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0003-2314-3453]{M. Pe\~na-Guerrero}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{N. Pirzkal}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2509-3878]{R. Plesha}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7617-5665]{C. R. Proffitt}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2662-6821]{B. J. Rauscher}$
$\affiliation{NASA Goddard Space Flight Center, Observational Cosmology Laboratory, Greenbelt, USA}$
$\author{H.-W. Rix }$
$\affiliation{Max-Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany}$
$\author[0000-0001-5171-3930]{B. Rodr\'iguez del Pino}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author{Z. Rustamkulov}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0003-2954-7643]{E. Sabbi}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-6050-7645]{D. K. Sing}$
$\affil{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author{M. Sirianni}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{M. te Plate}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7130-2880]{L. \'Ubeda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6570-4776]{G. M. Wahlgren}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{E. Wislowski}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{R. Wu}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4201-7367]{Chris J. Willott}$
$\affil{NRC Herzberg, 5071 West Saanich Rd, Victoria, BC V9E 2E7, Canada}$
$\begin{abstract}$
$The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectrophotometric and wavelength calibration of NIRSpec spectra, and new the 'as measured' sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs.$
$\end{abstract}$
$\keywords{Instrumentation: spectrographs - Space vehicles: instruments}$
$\n\end{document}\end{figure}}\end{figure}}\end{enumerate}}\end{itemize}}$
$\newcommand{$\eit$}{\end{itemize}}$
$\newcommand{\ben}{\begin{enumerate}}$
$\newcommand{$\een$}{\end{enumerate}}$
$\newcommand{\bfi}{\begin{figure}[htb]}$
$\newcommand{\bpfi}{\begin{figure}[p]}$
$\newcommand{$\lbol$}{\rm L_{Bol}}$
$\newcommand{$\llyc$}{\rm L_{Lyc}}$
$\newcommand{$\lk$}{\rm L_K}$
$\newcommand{$\av$}{\rm A_V}$
$\newcommand{$\m$v}{\rm M_V}$
$\newcommand{$\m$b}{\rm M_B}$
$\newcommand{$\htwo$}{\rm H_2}$
$\newcommand{$\fetwo$}{\rm[Fe II]}$
$\newcommand{$\lir$}{\rm L_{IR}}$
$\newcommand{$\brg$}{\rm Br\gamma}$
$\newcommand{$\ha$}{\rm H\alpha}$
$\newcommand{$\hb$}{\rm H\beta}$
$\newcommand{$\hel$}{\rm He I}$
$\newcommand{$\paa$}{\rm Pa\alpha}$
$\newcommand{$\m$htwo}{M_{\rm H_2}}$
$\newcommand{$\htwo$line}{\rm H_2 (1-0) S1}$
$\begin{document}$
$\title{In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the James Webb Space Telescope}$
$\author[0000-0002-5666-7782]{T. B\"oker}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6881-0574]{T. L. Beck}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7058-1726]{S. M. Birkmann}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-9262-7155]{G. Giardino}$
$\affiliation{ATG Europe for the European Space Agency, ESTEC, Noordwijk, The Netherlands}$
$\author[0000-0002-4834-369X]{C. Keyes}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-5320-2568]{N. Kumari}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{J. Muzerolle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-7028-5588]{T. Rawle}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6091-7924]{P. Zeidler}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{Y. Abul-Huda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2896-4138]{C. Alves de Oliveira}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0001-7997-1640]{S. Arribas}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0002-7722-6900]{K. Bechtold}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-0883-2226]{R. Bhatawdekar}$
$\affiliation{European Space Agency, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands}$
$\author[0000-0001-8470-7094]{N. Bonaventura}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{A. J. Bunker}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-0450-7306]{A. J. Cameron}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-6719-380X]{S. Carniani}$
$\affiliation{Scuola Normale Superiore, Piazza dei Cavalieri 7, I-56126 Pisa, Italy}$
$\author[0000-0003-3458-2275]{S. Charlot}$
$\affiliation{Sorbonne Universit\'e, UPMC-CNRS, UMR7095, Institut d'Astrophysique de Paris, F-75014 Paris, France}$
$\author[0000-0002-2678-2560]{M. Curti}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0001-9513-1449]{N. Espinoza}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\affiliation{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0001-8895-0606]{P. Ferruit}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0002-8871-3027]{M. Franx}$
$\affiliation{Leiden Observatory, Leiden University, PO Box 9513, 2300RA Leiden, The Netherlands}$
$\author[0000-0002-6780-2441]{P. Jakobsen}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{D. Karakla}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1016-9283]{ M.~López-Caniego}$
$\affiliation{Aurora Technology for the European Space Agency, Villanueva de la Ca\~nada, E-28692 Madrid, Spain.}$
$\affiliation{Universidad Europea de Madrid, 28670, Madrid, Spain.}$
$\author[0000-0002-4034-0080]{N. L\"utzgendorf}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4985-3819]{R. Maiolino}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0003-0192-6887]{E. Manjavacas}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-5788-5258]{A. P. Marston}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author{S. H. Moseley}$
$\affiliation{Quantum Circuits, Inc., New Haven, Connecticut, USA}$
$\author[0000-0002-3471-981X]{P. Ogle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0362-5941]{M. Perna}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0003-2314-3453]{M. Pe\~na-Guerrero}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{N. Pirzkal}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2509-3878]{R. Plesha}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7617-5665]{C. R. Proffitt}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2662-6821]{B. J. Rauscher}$
$\affiliation{NASA Goddard Space Flight Center, Observational Cosmology Laboratory, Greenbelt, USA}$
$\author{H.-W. Rix }$
$\affiliation{Max-Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany}$
$\author[0000-0001-5171-3930]{B. Rodr\'iguez del Pino}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author{Z. Rustamkulov}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0003-2954-7643]{E. Sabbi}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-6050-7645]{D. K. Sing}$
$\affil{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author{M. Sirianni}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{M. te Plate}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7130-2880]{L. \'Ubeda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6570-4776]{G. M. Wahlgren}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{E. Wislowski}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{R. Wu}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4201-7367]{Chris J. Willott}$
$\affil{NRC Herzberg, 5071 West Saanich Rd, Victoria, BC V9E 2E7, Canada}$
$\begin{abstract}$
$The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectrophotometric and wavelength calibration of NIRSpec spectra, and new the 'as measured' sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs.$
$\end{abstract}$
$\keywords{Instrumentation: spectrographs - Space vehicles: instruments}$
$\n\end{document}\end{figure}}\end{figure}}\end{enumerate}}$
$\newcommand{$\een$}{\end{enumerate}}$
$\newcommand{\bfi}{\begin{figure}[htb]}$
$\newcommand{\bpfi}{\begin{figure}[p]}$
$\newcommand{$\lbol$}{\rm L_{Bol}}$
$\newcommand{$\llyc$}{\rm L_{Lyc}}$
$\newcommand{$\lk$}{\rm L_K}$
$\newcommand{$\av$}{\rm A_V}$
$\newcommand{$\m$v}{\rm M_V}$
$\newcommand{$\m$b}{\rm M_B}$
$\newcommand{$\htwo$}{\rm H_2}$
$\newcommand{$\fetwo$}{\rm[Fe II]}$
$\newcommand{$\lir$}{\rm L_{IR}}$
$\newcommand{$\brg$}{\rm Br\gamma}$
$\newcommand{$\ha$}{\rm H\alpha}$
$\newcommand{$\hb$}{\rm H\beta}$
$\newcommand{$\hel$}{\rm He I}$
$\newcommand{$\paa$}{\rm Pa\alpha}$
$\newcommand{$\m$htwo}{M_{\rm H_2}}$
$\newcommand{$\htwo$line}{\rm H_2 (1-0) S1}$
$\begin{document}$
$\title{In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the James Webb Space Telescope}$
$\author[0000-0002-5666-7782]{T. B\"oker}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6881-0574]{T. L. Beck}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7058-1726]{S. M. Birkmann}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-9262-7155]{G. Giardino}$
$\affiliation{ATG Europe for the European Space Agency, ESTEC, Noordwijk, The Netherlands}$
$\author[0000-0002-4834-369X]{C. Keyes}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-5320-2568]{N. Kumari}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{J. Muzerolle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-7028-5588]{T. Rawle}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6091-7924]{P. Zeidler}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{Y. Abul-Huda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2896-4138]{C. Alves de Oliveira}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0001-7997-1640]{S. Arribas}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0002-7722-6900]{K. Bechtold}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-0883-2226]{R. Bhatawdekar}$
$\affiliation{European Space Agency, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands}$
$\author[0000-0001-8470-7094]{N. Bonaventura}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{A. J. Bunker}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-0450-7306]{A. J. Cameron}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-6719-380X]{S. Carniani}$
$\affiliation{Scuola Normale Superiore, Piazza dei Cavalieri 7, I-56126 Pisa, Italy}$
$\author[0000-0003-3458-2275]{S. Charlot}$
$\affiliation{Sorbonne Universit\'e, UPMC-CNRS, UMR7095, Institut d'Astrophysique de Paris, F-75014 Paris, France}$
$\author[0000-0002-2678-2560]{M. Curti}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0001-9513-1449]{N. Espinoza}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\affiliation{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0001-8895-0606]{P. Ferruit}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0002-8871-3027]{M. Franx}$
$\affiliation{Leiden Observatory, Leiden University, PO Box 9513, 2300RA Leiden, The Netherlands}$
$\author[0000-0002-6780-2441]{P. Jakobsen}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{D. Karakla}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1016-9283]{ M.~López-Caniego}$
$\affiliation{Aurora Technology for the European Space Agency, Villanueva de la Ca\~nada, E-28692 Madrid, Spain.}$
$\affiliation{Universidad Europea de Madrid, 28670, Madrid, Spain.}$
$\author[0000-0002-4034-0080]{N. L\"utzgendorf}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4985-3819]{R. Maiolino}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0003-0192-6887]{E. Manjavacas}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-5788-5258]{A. P. Marston}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author{S. H. Moseley}$
$\affiliation{Quantum Circuits, Inc., New Haven, Connecticut, USA}$
$\author[0000-0002-3471-981X]{P. Ogle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0362-5941]{M. Perna}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0003-2314-3453]{M. Pe\~na-Guerrero}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{N. Pirzkal}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2509-3878]{R. Plesha}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7617-5665]{C. R. Proffitt}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2662-6821]{B. J. Rauscher}$
$\affiliation{NASA Goddard Space Flight Center, Observational Cosmology Laboratory, Greenbelt, USA}$
$\author{H.-W. Rix }$
$\affiliation{Max-Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany}$
$\author[0000-0001-5171-3930]{B. Rodr\'iguez del Pino}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author{Z. Rustamkulov}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0003-2954-7643]{E. Sabbi}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-6050-7645]{D. K. Sing}$
$\affil{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author{M. Sirianni}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{M. te Plate}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7130-2880]{L. \'Ubeda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6570-4776]{G. M. Wahlgren}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{E. Wislowski}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{R. Wu}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4201-7367]{Chris J. Willott}$
$\affil{NRC Herzberg, 5071 West Saanich Rd, Victoria, BC V9E 2E7, Canada}$
$\begin{abstract}$
$The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectrophotometric and wavelength calibration of NIRSpec spectra, and new the 'as measured' sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs.$
$\end{abstract}$
$\keywords{Instrumentation: spectrographs - Space vehicles: instruments}$
$\n\end{document}\end{figure}}\end{figure}}$
$\newcommand{\bpfi}{\begin{figure}[p]}$
$\newcommand{$\lbol$}{\rm L_{Bol}}$
$\newcommand{$\llyc$}{\rm L_{Lyc}}$
$\newcommand{$\lk$}{\rm L_K}$
$\newcommand{$\av$}{\rm A_V}$
$\newcommand{$\m$v}{\rm M_V}$
$\newcommand{$\m$b}{\rm M_B}$
$\newcommand{$\htwo$}{\rm H_2}$
$\newcommand{$\fetwo$}{\rm[Fe II]}$
$\newcommand{$\lir$}{\rm L_{IR}}$
$\newcommand{$\brg$}{\rm Br\gamma}$
$\newcommand{$\ha$}{\rm H\alpha}$
$\newcommand{$\hb$}{\rm H\beta}$
$\newcommand{$\hel$}{\rm He I}$
$\newcommand{$\paa$}{\rm Pa\alpha}$
$\newcommand{$\m$htwo}{M_{\rm H_2}}$
$\newcommand{$\htwo$line}{\rm H_2 (1-0) S1}$
$\begin{document}$
$\title{In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the James Webb Space Telescope}$
$\author[0000-0002-5666-7782]{T. B\"oker}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6881-0574]{T. L. Beck}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7058-1726]{S. M. Birkmann}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-9262-7155]{G. Giardino}$
$\affiliation{ATG Europe for the European Space Agency, ESTEC, Noordwijk, The Netherlands}$
$\author[0000-0002-4834-369X]{C. Keyes}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-5320-2568]{N. Kumari}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{J. Muzerolle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-7028-5588]{T. Rawle}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6091-7924]{P. Zeidler}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{Y. Abul-Huda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2896-4138]{C. Alves de Oliveira}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0001-7997-1640]{S. Arribas}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0002-7722-6900]{K. Bechtold}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-0883-2226]{R. Bhatawdekar}$
$\affiliation{European Space Agency, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands}$
$\author[0000-0001-8470-7094]{N. Bonaventura}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{A. J. Bunker}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-0450-7306]{A. J. Cameron}$
$\affiliation{Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK}$
$\author[0000-0002-6719-380X]{S. Carniani}$
$\affiliation{Scuola Normale Superiore, Piazza dei Cavalieri 7, I-56126 Pisa, Italy}$
$\author[0000-0003-3458-2275]{S. Charlot}$
$\affiliation{Sorbonne Universit\'e, UPMC-CNRS, UMR7095, Institut d'Astrophysique de Paris, F-75014 Paris, France}$
$\author[0000-0002-2678-2560]{M. Curti}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0001-9513-1449]{N. Espinoza}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\affiliation{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0001-8895-0606]{P. Ferruit}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author[0000-0002-8871-3027]{M. Franx}$
$\affiliation{Leiden Observatory, Leiden University, PO Box 9513, 2300RA Leiden, The Netherlands}$
$\author[0000-0002-6780-2441]{P. Jakobsen}$
$\affiliation{Cosmic Dawn Center (DAWN), Niels Bohr Institute, University of Copenhagen, Jagtvej 128, DK-2200, Denmark}$
$\author{D. Karakla}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-1016-9283]{ M.~López-Caniego}$
$\affiliation{Aurora Technology for the European Space Agency, Villanueva de la Ca\~nada, E-28692 Madrid, Spain.}$
$\affiliation{Universidad Europea de Madrid, 28670, Madrid, Spain.}$
$\author[0000-0002-4034-0080]{N. L\"utzgendorf}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4985-3819]{R. Maiolino}$
$\affiliation{Cavendish Laboratory, University of Cambridge, 19 J. J. Thomson Ave., Cambridge CB3 0HE, UK}$
$\affiliation{Kavli Institute for Cosmology, University of Cambridge, Madingley Road, Cambridge CB3 0HA, UK}$
$\author[0000-0003-0192-6887]{E. Manjavacas}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-5788-5258]{A. P. Marston}$
$\affiliation{European Space Agency, ESAC, Villanueva de la Cañada, E-28692 Madrid, Spain}$
$\author{S. H. Moseley}$
$\affiliation{Quantum Circuits, Inc., New Haven, Connecticut, USA}$
$\author[0000-0002-3471-981X]{P. Ogle}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-0362-5941]{M. Perna}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author[0000-0003-2314-3453]{M. Pe\~na-Guerrero}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{N. Pirzkal}$
$\affiliation{AURA for the European Space Agency,$
$Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-2509-3878]{R. Plesha}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7617-5665]{C. R. Proffitt}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0003-2662-6821]{B. J. Rauscher}$
$\affiliation{NASA Goddard Space Flight Center, Observational Cosmology Laboratory, Greenbelt, USA}$
$\author{H.-W. Rix }$
$\affiliation{Max-Planck Institute for Astronomy, Königstuhl 17, 69117 Heidelberg, Germany}$
$\author[0000-0001-5171-3930]{B. Rodr\'iguez del Pino}$
$\affiliation{Centro de Astrobiolog\'ia (CAB), CSIC–INTA, Cra. de Ajalvir Km.~4, 28850- Torrej\'on de Ardoz, Madrid, Spain}$
$\author{Z. Rustamkulov}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author[0000-0003-2954-7643]{E. Sabbi}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-6050-7645]{D. K. Sing}$
$\affil{Department of Physics \& Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\affil{Department of Earth \& Planetary Sciences, Johns Hopkins University, Baltimore, MD 21218, USA}$
$\author{M. Sirianni}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{M. te Plate}$
$\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0001-7130-2880]{L. \'Ubeda}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-6570-4776]{G. M. Wahlgren}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{E. Wislowski}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author{R. Wu}$
$\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\author[0000-0002-4201-7367]{Chris J. Willott}$
$\affil{NRC Herzberg, 5071 West Saanich Rd, Victoria, BC V9E 2E7, Canada}$
$\begin{abstract}$
$The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectrophotometric and wavelength calibration of NIRSpec spectra, and new the 'as measured' sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs.$
$\end{abstract}$
$\keywords{Instrumentation: spectrographs - Space vehicles: instruments}$
$\n\end{document}\end{figure}}$
$\newcommand{$\lbol$}{\rm L_{Bol}}$
$\newcommand{$\llyc$}{\rm L_{Lyc}}$
$\newcommand{$\lk$}{\rm L_K}$
$\newcommand{$\av$}{\rm A_V}$
$\newcommand{$\m$v}{\rm M_V}$
$\newcommand{$\m$b}{\rm M_B}$
$\newcommand{$\htwo$}{\rm H_2}$
$\newcommand{$\fetwo$}{\rm[Fe II]}$
$\newcommand{$\lir$}{\rm L_{IR}}$
$\newcommand{$\brg$}{\rm Br\gamma}$
$\newcommand{$\ha$}{\rm H\alpha}$
$\newcommand{$\hb$}{\rm H\beta}$
$\newcommand{$\hel$}{\rm He I}$
$\newcommand{$\paa$}{\rm Pa\alpha}$
$\newcommand{$\m$htwo}{M_{\rm H_2}}$
$\newcommand{$\htwo$line}{\rm H_2 (1-0) S1}$
$\newcommand{$\arraystretch$}{1.2}$
$\newcommand{$\arraystretch$}{1.2}$
$\newcommand{$\arraystretch$}{1.2}$
$\newcommand{$\arraystretch$}{1.2}$</div>



<div id="title">

# In-orbit Performance of the Near-Infrared Spectrograph NIRSpec on the James Webb Space Telescope

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.13766-b31b1b.svg)](https://arxiv.org/abs/2301.13766)<mark>Appeared on: 2023-02-01</mark> - _accepted by PASP for special issue on JWST in-orbit performance_

</div>
<div id="authors">

T. B\"oker, et al. -- incl., <mark>H.-W. Rix </mark>

</div>
<div id="abstract">

**Abstract:** The Near-Infrared Spectrograph (NIRSpec) is one of the four focal plane instruments on the James Webb Space Telescope. In this paper, we summarize the in-orbit performance of NIRSpec, as derived from data collected during its commissioning campaign and the first few months of nominal science operations. More specifically, we discuss the performance of some critical hardware components such as the two NIRSpec Hawaii-2RG (H2RG) detectors, wheel mechanisms, and the micro-shutter array. We also summarize the accuracy of the two target acquisition procedures used to accurately place science targets into the slit apertures, discuss the current status of the spectrophotometric and wavelength calibration of NIRSpec spectra, and provide the 'as measured' sensitivity in all NIRSpec science modes. Finally, we point out a few important considerations for the preparation of NIRSpec science programs.

</div>

<div id="div_fig1">

<img src="tmp_2301.13766/./Figure_wata.png" alt="Fig8" width="100%"/>

**Figure 8. -** (a) The Wide Aperture Target Acquisition (WATA) exposure image (from PID 1118 Obs 3).  The 32 pixel by 32 pixel aperture is placed around the 1.$"$6 $\times$ 1.$"$6 (approximately 16 by 16 pixels) square aperture, but not exactly centered.  (b) The V2, V3 coordinate offset of the initial blind pointing position of the star that is calculated for the first exposure for each WATA Observation test.  The average blind pointing for the star is better than 50 mas radial from the wide aperture center. (c) The post-target acquisition pointing showing the improved centering of the TA target.  The average corrected position for the stars is better than 2.5 mas radial.  (*fig:wata*)

</div>
<div id="div_fig2">

<img src="tmp_2301.13766/./light_path_color.png" alt="Fig4.1" width="50%"/><img src="tmp_2301.13766/./nirspec_cad_color.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Left: The optical path through the NIRSpec instrument. The dispersion direction is out of the page. Right: CAD rendering of NIRSpec with its major mechanisms identified. The dimensions of the NIRSpec optical assembly are 1.9 m $\times$ 1.4 m $\times$ 0.7 m, with a total mass of 196 kg. (*fig:layout*)

</div>
<div id="div_fig3">

<img src="tmp_2301.13766/./Figure_msata_cycle1.png" alt="Fig10.1" width="50%"/><img src="tmp_2301.13766/./Figure_cycle1_ref.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** Left: computed offset correction (in V2/V3 and roll angle) computed by the onboard algorithm for the ensemble of 20 MSATA visits analyzed so far. Each star symbol contains the average over the $n$ reference stars used for the computation (color-coded by the value of $5 \leq n \leq 8$). Right: the results after execution of the corrective slew computed via the onboard MSATA algorithm, as measured from the subsequent reference image (which still has the TA filter in place).  (*fig:TAquality*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

187  publications in the last 7 days.
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers